# SuAVE Image to Label CNN Prediction Model

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 

<IPython.core.display.Javascript object>

### Import all packages (this might take a few seconds)

In [2]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Using TensorFlow backend.


## Initializing the number of times the model will loop and its batch size for learning

In [3]:
epochCount = {
    '25 Iterations': 25,
    '50 Iterations': 50,
    '75 Iterations': 75
}

def f(epoch_count):
    return epoch_count

epochNum = interact(f, epoch_count=epochCount)

interactive(children=(Dropdown(description='epoch_count', options={'25 Iterations': 25, '75 Iterations': 75, '…

In [4]:
batchS = {
    '32 Batch Size': 32,
    '64 Batch Size': 64,
    '128 Batch Size': 128
}

def f(batch_size):
    return batch_size

batchNum = interact(f, batch_size=batchS)

interactive(children=(Dropdown(description='batch_size', options={'64 Batch Size': 64, '32 Batch Size': 32, '1…

In [14]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = epochNum.widget.result
INIT_LR = 1e-3
BS = batchNum.widget.result
# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [15]:

# testing cell for Zeppelin Vanbarriger. Will be removed in production
survey_url = 'http://localhost:3001/main/file=Zeppelin-V-Alt_balthus_test.csv&views=1110000&view=grid'
params =''

# Testing Cells
csv_file = 'balthus.csv'


dzc_file = '/home/zep/WebstormProjects/SuAVE/app/public/imageserver/balthus/balthus.dzc'

dzc_url = 'http://localhost:3001/imageserver/balthus/balthus.dzc'

active_object = ''
actual_imageFolder = 'app/public/imagestore/balthus'
absolutePath = '/home/zep/WebstormProjects/SuAVE/app/public/imageserver/balthus/'

## Run and choose the column name that coresponds with the column label to be predicted

In [16]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)
# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu
# choose which label for predictions
out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'Media': 'Media', 'Location': 'Locatio…

## Grab the images and configure them for predicting

### This might take a little while depending on the size of the dataset

In [17]:
# grab chosen column names
nameCol = df['#img']
predCol = df[out2.widget.result]

# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])
    
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./Balthus/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

# Shuffle the data
p = np.random.permutation(len(yset))

# Relable for splitting sets
Y = []
X = []
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])
    
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

# parsed Y data containers
trainY = []
testY = []

# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")
# initialize the model
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

## Train the predictive model

### This is relative to the size of the data set and may take a few minutes 

In [18]:
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

Epoch 1/50
1/1 [==============================] - 1s 506ms/step - loss: 2.4314 - acc: 0.2500 - val_loss: 1.7133 - val_acc: 0.5417
Epoch 2/50
1/1 [==============================] - 1s 637ms/step - loss: 1.7968 - acc: 0.5312 - val_loss: 1.7602 - val_acc: 0.5417
Epoch 3/50
1/1 [==============================] - 0s 167ms/step - loss: 1.3021 - acc: 0.6250 - val_loss: 2.0345 - val_acc: 0.5417
Epoch 4/50
1/1 [==============================] - 1s 611ms/step - loss: 2.0190 - acc: 0.5469 - val_loss: 1.6932 - val_acc: 0.2083
Epoch 5/50
1/1 [==============================] - 0s 159ms/step - loss: 1.4911 - acc: 0.3750 - val_loss: 1.9963 - val_acc: 0.2083
Epoch 6/50
1/1 [==============================] - 1s 609ms/step - loss: 1.9975 - acc: 0.1562 - val_loss: 1.7592 - val_acc: 0.2083
Epoch 7/50
1/1 [==============================] - 0s 178ms/step - loss: 2.2540 - acc: 0.1250 - val_loss: 1.6787 - val_acc: 0.5417
Epoch 8/50
1/1 [==============================] - 1s 584ms/step - loss: 1.7389 - acc: 0.46

## Take the original data and predict based on the model

In [19]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

predictionsMade = []
preds = model.predict(img_check)

# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
    
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.5416666666666666


In [20]:
model.save('cnnModel.h5')

In [21]:
from keras.models import load_model
model2 = load_model('cnnModel.h5')

In [22]:
predictionsMade

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

## Enter a new header for the prediction column

In [37]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value=None,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value=None,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

Input new column Header Label: 


Text(value='', placeholder='Type label here')

Text(value='', placeholder='New Header will be displayed here')

## Write the predictions back to the original CSV

In [64]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred

# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)

In [65]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

Input survey name here:


A Jupyter Widget

A Jupyter Widget

In [66]:
import re
#Print survey name
survey_name = output_text.value
print("Survey name is:", survey_name)

Survey name is: Cyclops2


In [ ]:
import requests
upload_url = "http://suave.sdsc.edu:3001/uploadCSV"
upload_data = {'name': survey_name, 'dzc': dzc_url, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = "https://suave-dev.sdsc.edu/main/file=" + user + "_" + survey_url + ".csv" + "&views=1110001&view=grid"
print(url)

In [55]:
import requests
upload_url = "http://localhost:3001/uploadCSV"
upload_data = {'name': survey_name, 'dzc': dzc_url, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = "http://localhost:3001/main/file=" + user + "_" + survey_url + ".csv" + "&views=111000&view=grid"
print(url)

ConnectionError: HTTPConnectionPool(host='localhost', port=3001): Max retries exceeded with url: /uploadCSV (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C8E6B1D588>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))